In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, utils, models, callbacks as cb

In [ ]:
image_size = (780,1280)
batch_size = 10

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "training",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "validation",
    validation_split=0.8,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
train_ds = train_ds.prefetch(buffer_size=10)
val_ds = val_ds.prefetch(buffer_size=10)
print(f"{type(train_ds)}")




In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(25, 45))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(int(labels[i]))
#         plt.axis("off")

In [ ]:
def make_model(input_shape, num_classes):
    return keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.7),
        layers.Dense(num_classes, activation="softmax"),
    ]
    )


model = make_model(input_shape=image_size + (3,), num_classes=5)

In [ ]:
epochs = 50

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
    keras.callbacks.TensorBoard(log_dir="./logs")
]

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks,validation_data = val_ds,use_multiprocessing=True,
)
model.save('chicken.h5')